In [1]:
import rebound 
import numpy as np
import ctypes
from ctypes import POINTER, c_double, c_int,byref

In [60]:
def move_to_hel(self):
    r = self.particles[0].xyz
    v = self.particles[0].vxyz
    for p in self.particles:
        p.x -= r[0]
        p.y -= r[1]
        p.z -= r[2]
        p.vx -= v[0]
        p.vy -= v[1]
        p.vz -= v[2]        
rebound.Simulation.move_to_hel = move_to_hel

In [2]:
mercury = ctypes.cdll.LoadLibrary("./mercury/mercury.so")

In [54]:
def mercury_step(self):
    # assume sim.G=1, sim.particles[0].m =1, sim.dt in yrs/2pi
    nbod = c_int(self.N)
    nbig = nbod
    algor = 1 # Mixed variable Symplectic
    NMESS = 1 # number of messages
    CMAX = 1 # number of close encounter minima monitored simultaneously 
    step = mercury.mdt_mvsp_
    step.argtypes = [
        POINTER(c_double), # real*8              time 
        POINTER(c_double), # real*8              tstart
        POINTER(c_double), # real*8              h0
        POINTER(c_double), # real*8              tol
        POINTER(c_double), # real*8              rmax
        POINTER(c_double*3), # real*8(3)           en
        POINTER(c_double*3), # real*8(3)           am
        POINTER(c_double*3), # real*8(3)           jcen
        POINTER(c_double), # real*8              rcen
        POINTER(c_int), # integer             nbod
        POINTER(c_int), # integer             nbig
        POINTER(c_double*nbod.value), # real*8(nbod)        m
        POINTER((c_double*3)*nbod.value), # real*8(3,nbod)      x
        POINTER((c_double*3)*nbod.value), # real*8(3,nbod)      v
        POINTER((c_double*3)*nbod.value), # real*8(3,nbod)      s
        POINTER(c_double*nbod.value), # real*8(nbod)        rphys
        POINTER(c_double*nbod.value), # real*8(nbod)        rcrit
        POINTER(c_double*nbod.value), # real*8(nbod)        rce
        POINTER(c_int), # integer(nbod)       stat
    # character*8(nbod)   id
        POINTER((c_double*4)*nbod.value), # real*8(4,nbod)      ngf
        POINTER(c_int), # integer             algor
        POINTER(c_int*8),# integer(8)          opt
        POINTER(c_int), # integer             dtflag
        POINTER(c_int), # integer             ngflag
        POINTER(c_int), # integer             opflag
        POINTER(c_int), # integer             colflag
        POINTER(c_int), # integer             nclo
        POINTER(c_int*CMAX), # integer(CMAX)       iclo
        POINTER(c_int*CMAX), # integer(CMAX)       jclo
        POINTER(c_double*CMAX), # real*8(CMAX)        dclo
        POINTER(c_double*CMAX), # real*8(CMAX)        tclo
        POINTER(c_double*CMAX), # real*8(CMAX)        ixvclo
        POINTER(c_double*CMAX), # real*8(CMAX)        jxvclo
    # character*80(3)     outfile
    # character*80(NMESS) mem
        POINTER(c_int*NMESS),# integer(NMESS)     lmem
        ]


    time = c_double(0.)
    tstart = c_double(0.)
    h0 = c_double(self.dt)
    tol = c_double(1e-6)
    rmax = c_double(100.)
    DoubleArray3 = c_double * 3
    en = DoubleArray3(0.,0.,0.)
    am = DoubleArray3(0.,0.,0.)
    jcen = DoubleArray3(0.,0.,0.)
    rcen = c_double(0.)
    DoubleArrayNBOD = c_double * nbod.value
    DoubleArray3NBOD = (c_double * 3)* nbod.value
    _m = []
    _x = []
    _v = []
    _s = []
    for p in self.particles:
        _m.append(p.m)
        _x.append(DoubleArray3(p.x,p.y,p.z))
        _v.append(DoubleArray3(p.vx,p.vy,p.vz))
        _s.append(DoubleArray3(0.,0.,0.))
    m = DoubleArrayNBOD(*_m)
    x = DoubleArray3NBOD(*_x)
    v = DoubleArray3NBOD(*_v)
    s = DoubleArray3NBOD(*_s)
    rphys = DoubleArrayNBOD(0.,0.)
    rcrit = DoubleArrayNBOD(0.,0.)
    rce = DoubleArrayNBOD(0.,0.)
    stat = c_int(0)
    DoubleArray4 = (c_double * 4)
    DoubleArray4NBOD = (c_double * 4) * nbod.value
    ngf = DoubleArray4NBOD(DoubleArray4(0,0,0,0),DoubleArray4(0,0,0,0))
    algor = c_int(1)
    IntArray8 = c_int*8
    opt = IntArray8(0,1,1,2,0,1,0,0)
    dtflag = c_int(0)
    ngflag = c_int(0)
    opflag = c_int(0)
    colflag = c_int(0)
    nclo = c_int(1)
    iclo = (c_int*1)(0)
    jclo = (c_int*1)(0)
    dclo = (c_double*1)(0.)
    tclo = (c_double*1)(0.)
    ixvclo = (c_double*1)(0.)
    jxvclo = (c_double*1)(0.)
    lmem = (c_int*1)(0)

    step(byref(time), byref(tstart), byref(h0), byref(tol), byref(rmax), byref(en), byref(am), byref(jcen), byref(rcen),
         byref(nbod), byref(nbig), byref(m), byref(x), byref(v), byref(s), byref(rphys), byref(rcrit), byref(rce), byref(stat),
         byref(ngf), byref(algor), byref(opt), byref(dtflag), byref(ngflag), byref(opflag), byref(colflag), byref(nclo),
         byref(iclo), byref(jclo), byref(dclo), byref(tclo), byref(ixvclo), byref(jxvclo), byref(lmem))
    for i,p in enumerate(self.particles):
        self.particles[i].x = x[i][0]
        self.particles[i].y = x[i][1]
        self.particles[i].z = x[i][2]
        self.particles[i].vx = v[i][0]
        self.particles[i].vy = v[i][1]
        self.particles[i].vz = v[i][2]
    self.t += self.dt
rebound.Simulation.mercury_step = mercury_step

In [55]:
def getsim():
    sim = rebound.Simulation()
    sim.add(m=1)
    sim.add(a=1,m=1e-3)
    sim.integrator = "whfast"
    sim.dt = 0.1
    return sim

In [62]:
sim = getsim()
sim.mercury_step()
sim.move_to_hel()
sim.status()

---------------------------------
REBOUND version:     	3.6.6
REBOUND built on:    	Aug 20 2018 18:28:37
Number of particles: 	2
Selected integrator: 	whfast
Simulation time:     	1.0000000000000001e-01
Current timestep:    	0.100000
---------------------------------
<rebound.Particle object, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.001 x=0.9949991736113559 y=0.09988315429900474 z=0.0 vx=-0.09993308339699877 vy=0.9954965488854135 vz=0.0>
---------------------------------


In [63]:
sim = getsim()
sim.step()
sim.move_to_hel()
sim.status()

---------------------------------
REBOUND version:     	3.6.6
REBOUND built on:    	Aug 20 2018 18:28:37
Number of particles: 	2
Selected integrator: 	whfast
Simulation time:     	1.0000000000000001e-01
Current timestep:    	0.100000
---------------------------------
<rebound.Particle object, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.001 x=0.9949991736113559 y=0.09988315429900477 z=0.0 vx=-0.09993308339699875 vy=0.9954965488854135 vz=0.0>
---------------------------------
